In [ ]:
# --------------------------- Imports --------------------------- #
import os
import cv2
import torch
import numpy as np
import requests
from types import SimpleNamespace
from datetime import datetime
from geopy.distance import geodesic
import copy
import pandas as pd
import re
import scipy.ndimage
from ultralytics import YOLO
from RAFT.core.raft import RAFT
from RAFT.core.utils.utils import InputPadder

# --------------------------- Settings --------------------------- #
video_path =  #insert path
output_path = #insert path
yolo_model = #YOLO("best.pt")
raft_model_path = #"RAFT/models/raft-sintel.pth"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
interval = 10  # Process every 10th frame

# --------------------------- Load RAFT --------------------------- #
args = SimpleNamespace(small=False, mixed_precision=False, alternate_corr=False)
raft = RAFT(args)

def strip_module_prefix(state_dict):
    return {k.replace("module.", ""): v for k, v in state_dict.items()}

raft.load_state_dict(strip_module_prefix(torch.load(raft_model_path, map_location=DEVICE)))
raft.to(DEVICE)
raft.eval()

# --------------------------- Camera Data Helper --------------------------- #
def get_camera_json(request_timestamp):
    URL = 'https://cameras.alertcalifornia.org/public-camera-data/all_cameras-v3.json?rqts='
    unix_timestamp = int(request_timestamp.timestamp())
    URL += str(unix_timestamp)
    response = requests.get(URL)
    return response.json()

# --------------------------- Load Video --------------------------- #
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps * 0.25, (w, h))  # Slow motion output

# --------------------------- Get Camera FOV Info --------------------------- #
now = datetime.now()
camera_data = get_camera_json(now)

# Assuming you know your camera axis name, e.g., "axis-eaton"
camera_axis_name = "Axis-AlabamaHills1"

# Find the camera from the API response
camera_info = next((cam for cam in camera_data['features'] if cam['properties']['id'].lower() == camera_axis_name.lower()), None)
if camera_info is None:
    raise ValueError(f"Camera {camera_axis_name} not found!")

props = camera_info['properties']

# Swap fov_lft and fov_rt from (lon, lat) ➔ (lat, lon)
left_coord = (props['fov_lft'][1], props['fov_lft'][0])  # (lat, lon)
right_coord = (props['fov_rt'][1], props['fov_rt'][0])   # (lat, lon)

# Calculate real-world width across the frame
fov_width_meters = geodesic(left_coord, right_coord).meters

# Meters per pixel
meters_per_pixel = fov_width_meters / w

print(f"FOV width: {fov_width_meters:.2f} meters")
print(f"Meters per pixel: {meters_per_pixel:.6f}")

# --------------------------- Main Processing Loop --------------------------- #
ret, prev_frame = cap.read()
frame_index = 0
# --------------------------- Setup Limit on Frames --------------------------- #
video_duration_seconds = 20 # <-- Only process first 10 seconds
frame_limit = int(fps * video_duration_seconds)
print(f"Processing only first {frame_limit} frames (~{video_duration_seconds} seconds)")

while ret:
    ret, frame = cap.read()
    if not ret:
        break

    frame_index += 1

    if frame_index > frame_limit:
        print(f"Reached frame limit ({frame_limit}), stopping early.")
        break

    if frame_index % interval == 0:
        frame2 = frame.copy()
        frame1 = prev_frame.copy()

        results = yolo_model(frame2, verbose=False)[0]
        boxes = [box.xyxy[0].int().tolist() for box in results.boxes if box.conf > 0.05]
        print(f"[Frame {frame_index}] Detected {len(boxes)} raw objects.")

        merged_boxes = merge_boxes(boxes, iou_threshold=0.3)
        print(f"[Frame {frame_index}] Merged into {len(merged_boxes)} consolidated objects.")

        for x1, y1, x2, y2 in merged_boxes:
            crop1 = frame1[y1:y2, x1:x2]
            crop2 = frame2[y1:y2, x1:x2]

            if crop1.shape[0] < 10 or crop1.shape[1] < 10:
                continue

            image1 = torch.from_numpy(crop1).permute(2, 0, 1).float()[None].to(DEVICE) / 255.0
            image2 = torch.from_numpy(crop2).permute(2, 0, 1).float()[None].to(DEVICE) / 255.0

            padder = InputPadder(image1.shape)
            image1, image2 = padder.pad(image1, image2)

            with torch.no_grad():
                _, flow_up = raft(image1, image2, iters=20, test_mode=True)

            flow = flow_up[0].permute(1, 2, 0).cpu().numpy()
            flow = cv2.resize(flow, (x2 - x1, y2 - y1), interpolation=cv2.INTER_LINEAR)

            step = 10
            h_crop, w_crop = flow.shape[:2]
            yy, xx = np.mgrid[step//2:h_crop:step, step//2:w_crop:step]
            fx = flow[yy, xx, 0]
            fy = flow[yy, xx, 1]

            flow_magnitude = np.sqrt(fx**2 + fy**2)
            flow_magnitude_filtered = scipy.ndimage.median_filter(flow_magnitude, size=3)

            mean_pixel_movement = np.mean(flow_magnitude_filtered)
            mean_meter_movement = mean_pixel_movement * meters_per_pixel
            time_seconds = interval / fps
            smoke_speed_mps = mean_meter_movement / time_seconds

            label = f"Speed: {smoke_speed_mps:.2f} m/s"
            cv2.rectangle(frame2, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame2, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (255, 255, 255), 2, cv2.LINE_AA)

            for i in range(len(xx.flat)):
                pt1 = (x1 + int(xx.flat[i]), y1 + int(yy.flat[i]))
                pt2 = (x1 + int(xx.flat[i] + fx.flat[i]), y1 + int(yy.flat[i] + fy.flat[i]))
                cv2.arrowedLine(frame2, pt1, pt2, (0, 255, 0), 1, tipLength=0.3)

        out.write(frame2)
        cv2.imshow("RAFT Flow Overlay", frame2)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    prev_frame = frame.copy()


# --------------------------- Cleanup --------------------------- #
cap.release()
out.release()
cv2.destroyAllWindows()


FOV width: 33380.05 meters
Meters per pixel: 26.078162
Processing only first 240 frames (~20 seconds)
[Frame 10] Detected 1 raw objects.
[Frame 10] Merged into 1 consolidated objects.
[Frame 20] Detected 5 raw objects.
[Frame 20] Merged into 1 consolidated objects.
[Frame 30] Detected 3 raw objects.
[Frame 30] Merged into 1 consolidated objects.
[Frame 40] Detected 3 raw objects.
[Frame 40] Merged into 1 consolidated objects.
[Frame 50] Detected 2 raw objects.
[Frame 50] Merged into 1 consolidated objects.
[Frame 60] Detected 3 raw objects.
[Frame 60] Merged into 1 consolidated objects.
[Frame 70] Detected 3 raw objects.
[Frame 70] Merged into 2 consolidated objects.
[Frame 80] Detected 4 raw objects.
[Frame 80] Merged into 1 consolidated objects.
[Frame 90] Detected 2 raw objects.
[Frame 90] Merged into 1 consolidated objects.
[Frame 100] Detected 3 raw objects.
[Frame 100] Merged into 1 consolidated objects.
[Frame 110] Detected 3 raw objects.
[Frame 110] Merged into 1 consolidated o

In [15]:
def compute_iou(box1, box2):
    # box = [x1, y1, x2, y2]
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - inter_area

    if union_area == 0:
        return 0.0
    else:
        return inter_area / union_area


In [16]:
def merge_boxes(boxes, iou_threshold=0.3):
    merged = []
    used = [False] * len(boxes)

    for i in range(len(boxes)):
        if used[i]:
            continue
        # Start a new cluster
        current = boxes[i]
        for j in range(i + 1, len(boxes)):
            if used[j]:
                continue
            if compute_iou(current, boxes[j]) > iou_threshold:
                # Merge boxes: union of areas
                x1 = min(current[0], boxes[j][0])
                y1 = min(current[1], boxes[j][1])
                x2 = max(current[2], boxes[j][2])
                y2 = max(current[3], boxes[j][3])
                current = [x1, y1, x2, y2]
                used[j] = True
        merged.append(current)
    return merged


In [7]:
print(camera_info['properties'])


{'id': 'Axis-Alpine', 'name': '', 'last_frame_ts': 1745892544, 'fov_lft': [None, None], 'fov_rt': [None, None], 'fov_center': [None, None], 'fov': '', 'ProdNbr': None, 'az_current': 151.42999267578125, 'tilt_current': 0.0, 'zoom_current': 1.0, 'is_patrol_mode': 0, 'is_currently_patrolling': 0, 'state': '', 'county': '', 'isp': '', 'sponsor': '', 'region': ''}


In [11]:
def is_valid_coord(coord):
    if coord is None or len(coord) != 2:
        return False
    lat, lon = coord
    if lat is None or lon is None:
        return False
    if not (-90 <= lat <= 90):
        return False
    if not (-180 <= lon <= 180):
        return False
    return True

In [12]:
good_cameras = []
for feature in camera_data['features']:
    props = feature['properties']
    left = props.get('fov_lft')
    right = props.get('fov_rt')

    if left and right and is_valid_coord(left) and is_valid_coord(right):
        good_cameras.append(props['id'])

print(f"Found {len(good_cameras)} cameras with GOOD FOV!")
print(good_cameras)


Found 0 cameras with GOOD FOV!
[]


In [13]:
# Fetch latest cameras
now = datetime.now()
camera_data = get_camera_json(now)

# Pick first camera just as example
sample_camera = camera_data['features'][0]['properties']

# Print all keys and their values
for key, value in sample_camera.items():
    print(f"{key}: {value}")


id: AXIS-BCPOSLongmont
name: 
last_frame_ts: 1745892834
fov_lft: [None, None]
fov_rt: [None, None]
fov_center: [None, None]
fov: 
ProdNbr: None
az_current: 270.0
tilt_current: -0.6800000071525574
zoom_current: 1.0
is_patrol_mode: 0
is_currently_patrolling: 0
state: 
county: 
isp: 
sponsor: 
region: 
